In [344]:
import pandas as pd
import numpy as np
import datetime as dt
#hello yanjun
# read data
air_reserve = pd.read_csv('Data/air_reserve.csv')
air_store_info = pd.read_csv('Data/air_store_info.csv')
air_visit = pd.read_csv('Data/air_visit_data.csv')
holiday_info = pd.read_csv('Data/date_info.csv')
hpg_reserve = pd.read_csv('Data/hpg_reserve.csv')
hpg_store_info = pd.read_csv('Data/hpg_store_info.csv')
store_id_relation = pd.read_csv('Data/store_id_relation.csv')
sample_submission = pd.read_csv('Data/sample_submission.csv')

In [345]:
del hpg_store_info['latitude'], hpg_store_info['longitude']
hpg_store_info.head(5)

,hpg_store_id,hpg_genre_name,hpg_area_name
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō


In [346]:
air_store_info.head(5)

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [347]:
len(set(air_store_info['air_store_id']))

829

In [348]:
holiday_info['calendar_date'] = pd.to_datetime(holiday_info['calendar_date'])
holiday_info.columns = ['visit_datetime','day_of_week','holiday_flg']

holiday_info.head(5)

,visit_datetime,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [349]:
air_visit['visit_date'] = pd.to_datetime(air_visit['visit_date'])
air_visit.columns = ['air_store_id','visit_datetime','visitors']

air_visit.head(5)

,air_store_id,visit_datetime,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [350]:
sample_submission['calendar_date'] = sample_submission['id'].map(lambda x : x.split('_')[-1])
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x : x.split('_')[0] + '_' + x.split('_')[1])
sample_submission['calendar_date'] = pd.to_datetime(sample_submission['calendar_date'])
sample_submission['dow'] = sample_submission['calendar_date'].dt.dayofweek
sample_submission['month'] = sample_submission['calendar_date'].dt.month
sample_submission['year'] = sample_submission['calendar_date'].dt.year
sample_submission = sample_submission.rename(columns={'calendar_date':'visit_datetime'})
sample_submission = sample_submission.merge(holiday_info,how='left',on='visit_datetime')
sample_submission = sample_submission.merge(air_store_info,how='left',on='air_store_id')

sample_submission.head(5)

,id,visitors,visit_datetime,air_store_id,dow,month,year,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,4,2017,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,4,2017,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,4,2017,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,4,2017,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,4,2017,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595


In [351]:
weekends = ['Saturday','Sunday']
sample_submission['weekend_flg'] = ''
for index,row in sample_submission.iterrows():
    if row['day_of_week'] in weekends:
        sample_submission.set_value(index,'weekend_flg',1)
    else:
        sample_submission.set_value(index,'weekend_flg',0)
sample_submission.head(5)

,id,visitors,visit_datetime,air_store_id,dow,month,year,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,weekend_flg
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,4,2017,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,1
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,4,2017,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,4,2017,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,4,2017,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,4,2017,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0


In [352]:
hpg_reserve_air = hpg_reserve[hpg_reserve['hpg_store_id'].isin(set(store_id_relation['hpg_store_id']))]
air_reserve_daily = air_reserve.groupby(['air_store_id','visit_datetime']).agg({'reserve_visitors':np.sum}).reset_index()
hpg_reserve_daily = hpg_reserve_air.groupby(['hpg_store_id','visit_datetime']).agg({'reserve_visitors':np.sum}).reset_index()

air_reserve_daily['visit_datetime'] = pd.to_datetime(air_reserve_daily['visit_datetime']).dt.date
hpg_reserve_daily['visit_datetime'] = pd.to_datetime(hpg_reserve_daily['visit_datetime']).dt.date
# del air_reserve,hpg_reserve,hpg_reserve_air

In [353]:
air_reserve_daily.head(5)

,air_store_id,visit_datetime,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


In [354]:
hpg_reserve_daily.head(5)

,hpg_store_id,visit_datetime,reserve_visitors
0,hpg_0183ec352e38903c,2016-01-09,2
1,hpg_0183ec352e38903c,2016-01-11,8
2,hpg_0183ec352e38903c,2016-01-14,11
3,hpg_0183ec352e38903c,2016-01-15,8
4,hpg_0183ec352e38903c,2016-01-18,9


In [355]:
# define preprocess funcition
def data_preprocess(X):
    if X.columns[0] == 'air_store_id':
        temp = X.merge(store_id_relation,how='left',on='air_store_id').fillna(-1)
        temp = temp.merge(hpg_store_info,how='left',on='hpg_store_id').fillna(-1)
    else:
        temp = X.merge(store_id_relation,how='left',on='hpg_store_id').fillna(-1)
        temp = temp.merge(hpg_store_info,how='left',on='hpg_store_id').fillna(-1)
    temp['visit_datetime'] = pd.to_datetime(pd.to_datetime(temp['visit_datetime']).dt.date)
    return temp

In [356]:
air_r = data_preprocess(air_reserve_daily)
hpg_r = data_preprocess(hpg_reserve_daily)

all_r = hpg_r.append(air_r)
all_r['dayofweek'] = all_r['visit_datetime'].dt.dayofweek
all_r['weekofyear'] = all_r['visit_datetime'].dt.week
all_r['month'] = all_r['visit_datetime'].dt.month
all_r['year'] = all_r['visit_datetime'].dt.year

all_r.head(5)

,air_store_id,hpg_area_name,hpg_genre_name,hpg_store_id,reserve_visitors,visit_datetime,dayofweek,weekofyear,month,year
0,air_cbe867adcf44e14f,Fukuoka-ken Kitakyūshū-shi Aoyama,Creation,hpg_0183ec352e38903c,2,2016-01-09,5,1,1,2016
1,air_cbe867adcf44e14f,Fukuoka-ken Kitakyūshū-shi Aoyama,Creation,hpg_0183ec352e38903c,8,2016-01-11,0,2,1,2016
2,air_cbe867adcf44e14f,Fukuoka-ken Kitakyūshū-shi Aoyama,Creation,hpg_0183ec352e38903c,11,2016-01-14,3,2,1,2016
3,air_cbe867adcf44e14f,Fukuoka-ken Kitakyūshū-shi Aoyama,Creation,hpg_0183ec352e38903c,8,2016-01-15,4,2,1,2016
4,air_cbe867adcf44e14f,Fukuoka-ken Kitakyūshū-shi Aoyama,Creation,hpg_0183ec352e38903c,9,2016-01-18,0,3,1,2016


In [357]:
air_visit_holiday = air_visit.merge(holiday_info,how='left',on='visit_datetime')

air_visit_holiday['weekend_flg'] = ''
for index,row in air_visit_holiday.iterrows():
    if row['day_of_week'] in weekends:
        air_visit_holiday.set_value(index,'weekend_flg',1)
    else:
        air_visit_holiday.set_value(index,'weekend_flg',0)

train = air_visit_holiday.copy()
def date_split(X):
    X['year'] = X['visit_datetime'].dt.year
    X['month'] = X['visit_datetime'].dt.month
    X['dow'] = X['visit_datetime'].dt.dayofweek

date_split(train)
train = train.merge(air_store_info,how='left',on='air_store_id')
train.head(5)

,air_store_id,visit_datetime,visitors,day_of_week,holiday_flg,weekend_flg,year,month,dow,air_genre_name,air_area_name,latitude,longitude
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,0,2016,1,2,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,0,2016,1,3,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,0,2016,1,4,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,1,2016,1,5,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,0,2016,1,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [358]:
mean_vis = train.groupby(['air_store_id','dow'],
                         as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_vis'})
max_vis = train.groupby(['air_store_id','dow'],
                         as_index=False)['visitors'].max().rename(columns={'visitors':'max_vis'})
min_vis = train.groupby(['air_store_id','dow'],
                         as_index=False)['visitors'].min().rename(columns={'visitors':'min_vis'})
median_vis = train.groupby(['air_store_id','dow'],
                         as_index=False)['visitors'].median().rename(columns={'visitors':'median_vis'})
quantile25_vis = pd.DataFrame(train.groupby(['air_store_id','dow'])['visitors'].quantile(0.25).reset_index().rename(columns={'visitors':'quantile25_vis'}))
quantile75_vis = pd.DataFrame(train.groupby(['air_store_id','dow'])['visitors'].quantile(0.75).reset_index().rename(columns={'visitors':'quantile75_vis'}))

In [359]:
def merge_data(X,a,b,c,d,e,f):
    X = X.merge(a,how='left',on=['air_store_id','dow'])
    X = X.merge(b,how='left',on=['air_store_id','dow'])
    X = X.merge(c,how='left',on=['air_store_id','dow'])
    X = X.merge(d,how='left',on=['air_store_id','dow'])
    X = X.merge(e,how='left',on=['air_store_id','dow'])
    X = X.merge(f,how='left',on=['air_store_id','dow'])
    
    return X

train = merge_data(train,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis)
train.isnull().values.any()

False

In [360]:
train.shape

(252108, 19)

In [361]:
train.head(5)

,air_store_id,visit_datetime,visitors,day_of_week,holiday_flg,weekend_flg,year,month,dow,air_genre_name,air_area_name,latitude,longitude,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,0,2016,1,2,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,23.843750,57,7,25.0,17.75,27.5
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,0,2016,1,3,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,20.292308,54,2,21.0,12.00,26.0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,0,2016,1,4,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,34.738462,61,4,35.0,27.00,43.0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,1,2016,1,5,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,27.651515,53,6,27.0,20.25,34.0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,0,2016,1,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,13.754386,34,2,12.0,8.00,20.0


In [362]:
train['hpg_flg'] = train['air_store_id'].isin(set(store_id_relation['air_store_id']))

In [363]:
reserve_visitors = all_r.groupby(['air_store_id','visit_datetime'])['reserve_visitors'].sum().reset_index()
reserve_visitors.head(5)
reserve_visitors['reserve_visitors'].describe()

count    42193.000000
mean        12.727443
std         21.434135
min          1.000000
25%          4.000000
50%          8.000000
75%         16.000000
max       2241.000000
Name: reserve_visitors, dtype: float64

In [364]:
train = train.merge(reserve_visitors,how='left',on=['air_store_id','visit_datetime']).fillna(-1)
train.head(5)

,air_store_id,visit_datetime,visitors,day_of_week,holiday_flg,weekend_flg,year,month,dow,air_genre_name,...,latitude,longitude,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,0,2016,1,2,Dining bar,...,35.658068,139.751599,23.843750,57,7,25.0,17.75,27.5,False,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,0,2016,1,3,Dining bar,...,35.658068,139.751599,20.292308,54,2,21.0,12.00,26.0,False,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,0,2016,1,4,Dining bar,...,35.658068,139.751599,34.738462,61,4,35.0,27.00,43.0,False,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,1,2016,1,5,Dining bar,...,35.658068,139.751599,27.651515,53,6,27.0,20.25,34.0,False,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,0,2016,1,0,Dining bar,...,35.658068,139.751599,13.754386,34,2,12.0,8.00,20.0,False,-1.0


In [365]:
test = sample_submission.copy()
test.head(5)

,id,visitors,visit_datetime,air_store_id,dow,month,year,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,weekend_flg
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,4,2017,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,1
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,4,2017,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,4,2017,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,4,2017,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,4,2017,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0


In [366]:
test = merge_data(test,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis)

In [367]:
print(test[test.isnull().any(axis=1)].head())

                                 id  visitors visit_datetime  \
39  air_0164b9927d20bcc3_2017-04-23         0     2017-04-23   
46  air_0164b9927d20bcc3_2017-04-30         0     2017-04-30   
53  air_0164b9927d20bcc3_2017-05-07         0     2017-05-07   
60  air_0164b9927d20bcc3_2017-05-14         0     2017-05-14   
67  air_0164b9927d20bcc3_2017-05-21         0     2017-05-21   

            air_store_id  dow  month  year day_of_week  holiday_flg  \
39  air_0164b9927d20bcc3    6      4  2017      Sunday            0   
46  air_0164b9927d20bcc3    6      4  2017      Sunday            0   
53  air_0164b9927d20bcc3    6      5  2017      Sunday            0   
60  air_0164b9927d20bcc3    6      5  2017      Sunday            0   
67  air_0164b9927d20bcc3    6      5  2017      Sunday            0   

    air_genre_name                 air_area_name   latitude   longitude  \
39  Italian/French  Tōkyō-to Minato-ku Shibakōen  35.658068  139.751599   
46  Italian/French  Tōkyō-to Minato-ku

In [368]:
test = test.fillna(-1)
test['hpg_flg'] = test['air_store_id'].isin(set(store_id_relation['air_store_id']))
print(test[test.isnull().any(axis=1)].head())

Empty DataFrame
Columns: [id, visitors, visit_datetime, air_store_id, dow, month, year, day_of_week, holiday_flg, air_genre_name, air_area_name, latitude, longitude, weekend_flg, mean_vis, max_vis, min_vis, median_vis, quantile25_vis, quantile75_vis, hpg_flg]
Index: []

[0 rows x 21 columns]


In [369]:
test = test.merge(reserve_visitors,how='left',on=['air_store_id','visit_datetime']).fillna(-1)

In [370]:
hpg_store_air = hpg_store_info[hpg_store_info['hpg_store_id'].isin(set(store_id_relation['hpg_store_id']))]
hpg_store_air = hpg_store_air.merge(store_id_relation,how='left',on='hpg_store_id')

from sklearn.preprocessing import LabelEncoder as lbl
le = lbl()
le.fit(hpg_store_air['hpg_store_id'])
hpg_store_air['hpg_store_id'] = le.transform(hpg_store_air['hpg_store_id'])

le.fit(hpg_store_air['hpg_genre_name'])
hpg_store_air['hpg_genre_name'] = le.transform(hpg_store_air['hpg_genre_name'])

le.fit(hpg_store_air['hpg_area_name'])
hpg_store_air['hpg_area_name'] = le.transform(hpg_store_air['hpg_area_name'])

hpg_store_air.head(5)

,hpg_store_id,hpg_genre_name,hpg_area_name,air_store_id
0,48,9,20,air_2aab19554f91ff82
1,58,9,29,air_258ad2619d7bff9a
2,1,9,7,air_c47aa7493b15f297
3,23,9,32,air_96005f79124e12bf
4,38,9,30,air_f2c5a1f24279c531


In [371]:
train = train.merge(hpg_store_air,how='left',on='air_store_id').fillna(-1)
test = test.merge(hpg_store_air,how='left',on='air_store_id').fillna(-1)

In [372]:
test.head(5)

,id,visitors,visit_datetime,air_store_id,dow,month,year,day_of_week,holiday_flg,air_genre_name,...,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors,hpg_store_id,hpg_genre_name,hpg_area_name
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,4,2017,Sunday,0,Italian/French,...,2.0,2.0,2.0,2.00,2.00,True,-1.0,-1.0,-1.0,-1.0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,4,2017,Monday,0,Italian/French,...,47.0,1.0,19.0,17.00,26.50,True,-1.0,-1.0,-1.0,-1.0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,4,2017,Tuesday,0,Italian/French,...,43.0,1.0,24.5,19.00,30.00,True,-1.0,-1.0,-1.0,-1.0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,4,2017,Wednesday,0,Italian/French,...,52.0,15.0,28.0,20.75,33.25,True,-1.0,-1.0,-1.0,-1.0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,4,2017,Thursday,0,Italian/French,...,47.0,15.0,30.0,24.25,34.00,True,-1.0,-1.0,-1.0,-1.0


In [373]:
test.shape
train.head(5)

,air_store_id,visit_datetime,visitors,day_of_week,holiday_flg,weekend_flg,year,month,dow,air_genre_name,...,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors,hpg_store_id,hpg_genre_name,hpg_area_name
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,0,2016,1,2,Dining bar,...,57,7,25.0,17.75,27.5,False,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,0,2016,1,3,Dining bar,...,54,2,21.0,12.00,26.0,False,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,0,2016,1,4,Dining bar,...,61,4,35.0,27.00,43.0,False,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,1,2016,1,5,Dining bar,...,53,6,27.0,20.25,34.0,False,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,0,2016,1,0,Dining bar,...,34,2,12.0,8.00,20.0,False,-1.0,-1.0,-1.0,-1.0


In [374]:
le.fit(train['air_store_id'])
train['coded_id'] = le.transform(train['air_store_id'])
test['coded_id'] = le.transform(test['air_store_id'])

le.fit(train['air_genre_name'])
train['coded_genre'] = le.transform(train['air_genre_name'])
test['coded_genre'] = le.transform(test['air_genre_name'])
             
le.fit(train['air_area_name'])
train['coded_area'] = le.transform(train['air_area_name'])
test['coded_area'] = le.transform(test['air_area_name'])

print(test[test.isnull().any(axis=1)].head())

Empty DataFrame
Columns: [id, visitors, visit_datetime, air_store_id, dow, month, year, day_of_week, holiday_flg, air_genre_name, air_area_name, latitude, longitude, weekend_flg, mean_vis, max_vis, min_vis, median_vis, quantile25_vis, quantile75_vis, hpg_flg, reserve_visitors, hpg_store_id, hpg_genre_name, hpg_area_name, coded_id, coded_genre, coded_area]
Index: []

[0 rows x 28 columns]


In [375]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32019 entries, 0 to 32018
Data columns (total 28 columns):
id                  32019 non-null object
visitors            32019 non-null int64
visit_datetime      32019 non-null datetime64[ns]
air_store_id        32019 non-null object
dow                 32019 non-null int64
month               32019 non-null int64
year                32019 non-null int64
day_of_week         32019 non-null object
holiday_flg         32019 non-null int64
air_genre_name      32019 non-null object
air_area_name       32019 non-null object
latitude            32019 non-null float64
longitude           32019 non-null float64
weekend_flg         32019 non-null int64
mean_vis            32019 non-null float64
max_vis             32019 non-null float64
min_vis             32019 non-null float64
median_vis          32019 non-null float64
quantile25_vis      32019 non-null float64
quantile75_vis      32019 non-null float64
hpg_flg             32019 non-null bool
r

In [376]:
train.head(5)

,air_store_id,visit_datetime,visitors,day_of_week,holiday_flg,weekend_flg,year,month,dow,air_genre_name,...,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors,hpg_store_id,hpg_genre_name,hpg_area_name,coded_id,coded_genre,coded_area
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,0,2016,1,2,Dining bar,...,17.75,27.5,False,-1.0,-1.0,-1.0,-1.0,603,4,62
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,0,2016,1,3,Dining bar,...,12.00,26.0,False,-1.0,-1.0,-1.0,-1.0,603,4,62
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,0,2016,1,4,Dining bar,...,27.00,43.0,False,-1.0,-1.0,-1.0,-1.0,603,4,62
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,1,2016,1,5,Dining bar,...,20.25,34.0,False,-1.0,-1.0,-1.0,-1.0,603,4,62
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,0,2016,1,0,Dining bar,...,8.00,20.0,False,-1.0,-1.0,-1.0,-1.0,603,4,62


In [377]:
train.head(5)['air_genre_name']

0    Dining bar
1    Dining bar
2    Dining bar
3    Dining bar
4    Dining bar
Name: air_genre_name, dtype: object

In [378]:
train.head(5)['air_area_name']

0    Tōkyō-to Minato-ku Shibakōen
1    Tōkyō-to Minato-ku Shibakōen
2    Tōkyō-to Minato-ku Shibakōen
3    Tōkyō-to Minato-ku Shibakōen
4    Tōkyō-to Minato-ku Shibakōen
Name: air_area_name, dtype: object

In [379]:
test.columns,train.columns

(Index(['id', 'visitors', 'visit_datetime', 'air_store_id', 'dow', 'month',
        'year', 'day_of_week', 'holiday_flg', 'air_genre_name', 'air_area_name',
        'latitude', 'longitude', 'weekend_flg', 'mean_vis', 'max_vis',
        'min_vis', 'median_vis', 'quantile25_vis', 'quantile75_vis', 'hpg_flg',
        'reserve_visitors', 'hpg_store_id', 'hpg_genre_name', 'hpg_area_name',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'),
 Index(['air_store_id', 'visit_datetime', 'visitors', 'day_of_week',
        'holiday_flg', 'weekend_flg', 'year', 'month', 'dow', 'air_genre_name',
        'air_area_name', 'latitude', 'longitude', 'mean_vis', 'max_vis',
        'min_vis', 'median_vis', 'quantile25_vis', 'quantile75_vis', 'hpg_flg',
        'reserve_visitors', 'hpg_store_id', 'hpg_genre_name', 'hpg_area_name',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'))

In [380]:
del test['id'],test['visitors'],test['visit_datetime'],test['air_store_id'],test['day_of_week'],test['air_genre_name'],test['air_area_name'],test['hpg_store_id'],test['hpg_genre_name'],test['hpg_area_name']
del train['air_store_id'],train['visit_datetime'],train['day_of_week'],train['air_genre_name'],train['air_area_name'],train['hpg_store_id'],train['hpg_genre_name'],train['hpg_area_name']

In [381]:
test.info(),train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32019 entries, 0 to 32018
Data columns (total 18 columns):
dow                 32019 non-null int64
month               32019 non-null int64
year                32019 non-null int64
holiday_flg         32019 non-null int64
latitude            32019 non-null float64
longitude           32019 non-null float64
weekend_flg         32019 non-null int64
mean_vis            32019 non-null float64
max_vis             32019 non-null float64
min_vis             32019 non-null float64
median_vis          32019 non-null float64
quantile25_vis      32019 non-null float64
quantile75_vis      32019 non-null float64
hpg_flg             32019 non-null bool
reserve_visitors    32019 non-null float64
coded_id            32019 non-null int64
coded_genre         32019 non-null int64
coded_area          32019 non-null int64
dtypes: bool(1), float64(9), int64(8)
memory usage: 4.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 252108 entries, 0 to 252107

(None, None)

In [382]:
train_x,train_y = train.iloc[:,1:],train.iloc[:,0]
test_x = test

train_x.columns,test_x.columns

(Index(['holiday_flg', 'weekend_flg', 'year', 'month', 'dow', 'latitude',
        'longitude', 'mean_vis', 'max_vis', 'min_vis', 'median_vis',
        'quantile25_vis', 'quantile75_vis', 'hpg_flg', 'reserve_visitors',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'),
 Index(['dow', 'month', 'year', 'holiday_flg', 'latitude', 'longitude',
        'weekend_flg', 'mean_vis', 'max_vis', 'min_vis', 'median_vis',
        'quantile25_vis', 'quantile75_vis', 'hpg_flg', 'reserve_visitors',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'))

In [383]:
train_x = train_x.reindex(columns=test_x.columns)

In [384]:
train_x.columns,test_x.columns

(Index(['dow', 'month', 'year', 'holiday_flg', 'latitude', 'longitude',
        'weekend_flg', 'mean_vis', 'max_vis', 'min_vis', 'median_vis',
        'quantile25_vis', 'quantile75_vis', 'hpg_flg', 'reserve_visitors',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'),
 Index(['dow', 'month', 'year', 'holiday_flg', 'latitude', 'longitude',
        'weekend_flg', 'mean_vis', 'max_vis', 'min_vis', 'median_vis',
        'quantile25_vis', 'quantile75_vis', 'hpg_flg', 'reserve_visitors',
        'coded_id', 'coded_genre', 'coded_area'],
       dtype='object'))

In [385]:
test_x.head(5)

,dow,month,year,holiday_flg,latitude,longitude,weekend_flg,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors,coded_id,coded_genre,coded_area
0,6,4,2017,0,35.694003,139.753595,1,2.000000,2.0,2.0,2.0,2.00,2.00,True,-1.0,0,6,44
1,0,4,2017,0,35.694003,139.753595,0,22.457143,47.0,1.0,19.0,17.00,26.50,True,-1.0,0,6,44
2,1,4,2017,0,35.694003,139.753595,0,24.350000,43.0,1.0,24.5,19.00,30.00,True,-1.0,0,6,44
3,2,4,2017,0,35.694003,139.753595,0,28.125000,52.0,15.0,28.0,20.75,33.25,True,-1.0,0,6,44
4,3,4,2017,0,35.694003,139.753595,0,29.868421,47.0,15.0,30.0,24.25,34.00,True,-1.0,0,6,44


In [386]:
train_x.head(50)

,dow,month,year,holiday_flg,latitude,longitude,weekend_flg,mean_vis,max_vis,min_vis,median_vis,quantile25_vis,quantile75_vis,hpg_flg,reserve_visitors,coded_id,coded_genre,coded_area
0,2,1,2016,0,35.658068,139.751599,0,23.843750,57,7,25.0,17.75,27.5,False,-1.0,603,4,62
1,3,1,2016,0,35.658068,139.751599,0,20.292308,54,2,21.0,12.00,26.0,False,-1.0,603,4,62
2,4,1,2016,0,35.658068,139.751599,0,34.738462,61,4,35.0,27.00,43.0,False,-1.0,603,4,62
3,5,1,2016,0,35.658068,139.751599,1,27.651515,53,6,27.0,20.25,34.0,False,-1.0,603,4,62
4,0,1,2016,0,35.658068,139.751599,0,13.754386,34,2,12.0,8.00,20.0,False,-1.0,603,4,62
5,1,1,2016,0,35.658068,139.751599,0,18.580645,35,5,19.0,14.00,23.0,False,-1.0,603,4,62
6,2,1,2016,0,35.658068,139.751599,0,23.843750,57,7,25.0,17.75,27.5,False,-1.0,603,4,62
7,3,1,2016,0,35.658068,139.751599,0,20.292308,54,2,21.0,12.00,26.0,False,-1.0,603,4,62
8,4,1,2016,0,35.658068,139.751599,0,34.738462,61,4,35.0,27.00,43.0,False,-1.0,603,4,62
9,5,1,2016,0,35.658068,139.751599,1,27.651515,53,6,27.0,20.25,34.0,False,-1.0,603,4,62


In [387]:
train_x.columns

Index(['dow', 'month', 'year', 'holiday_flg', 'latitude', 'longitude',
       'weekend_flg', 'mean_vis', 'max_vis', 'min_vis', 'median_vis',
       'quantile25_vis', 'quantile75_vis', 'hpg_flg', 'reserve_visitors',
       'coded_id', 'coded_genre', 'coded_area'],
      dtype='object')

In [388]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.05, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
def rmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        if predicted[x]<0 or real[x]<0: #check for negative values
            continue
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

In [ ]:
train_x.to_csv('train_x_noHPG.csv',index=False)
train_y.to_csv('train_y_noHPG.csv',index=False)
test_x.to_csv('test_x_noHPG.csv',index=False)
test_y.to_csv('test_y_noHPG.csv',index=False)

In [ ]:
import xgboost as xgb
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=8,
    n_jobs=4,
    learning_rate=0.01,
    n_estimators=1000,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=27,
    **boost_params)

xgb0.fit(train_x, np.log1p(train_y))
predict_y = xgb0.predict(test_x)


In [ ]:
#rmsle_score_xgb = rmsle(test_y,predict_y)

In [ ]:
prediction_new_XGB = np.expm1(predict_y)
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':prediction_new_XGB})
submission.to_csv('submission_new_XGB.csv',index=False)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(train_x,label=np.log1p(train_y))
dtest = xgb.DMatrix(test_x)

In [ ]:
dtrain.get_label()

In [ ]:
'''
def rmsle(preds,dtrain):
    labels = dtrain_get_label()
    preds = preds
    
    assert len(y) == len(y_pred)
    terms_to_sum = [(y_pred[i] - y[i]) ** 2.0 for i,pred in enumerate(y_pred)]
    
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

'''
param = {'max_depth':10,'eta':0.01,'silent':0,'objective':'reg:linear','eval_metric':'rmse',
         'subsample':0.8,'colsample_bytree':0.8, 'n_jobs':4}
num_round = 1000
gbm = xgb.train(param,dtrain,num_round)

In [ ]:
#gbm.fit(train_x_m,np.log1p(train_y_m))
prediction = np.expm1(gbm.predict(dtest))

In [ ]:
gbm.get_score()

In [ ]:
prediction

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':prediction})
submission.to_csv('submission_XGB.csv',index=False)

In [ ]:
pd.DataFrame(prediction).describe()
rmsle_score_xgb = rmsle(test_y,prediction)

In [ ]:
#submission = pd.DataFrame({'id':sample_submission['id'],
#                           'visitors':prediction_df['prediction']})
#submission.to_csv('submission_XGB.csv',index=False)

In [ ]:
from sklearn import *

In [ ]:
#GBR = ensemble.GradientBoostingRegressor(learning_rate=0.02,n_jobs=4,random_state=5,n_estimators=100,subsample=1, max_depth=10)
KNR = neighbors.KNeighborsRegressor(n_jobs=4, n_neighbors=10)

In [ ]:
#GBR.fit(train_x,np.log1p(train_y))

In [ ]:
#GBR.feature_importances_

In [ ]:
#train_x.columns

In [ ]:
#prediction_GBR = np.expm1(GBR.predict(test_x))

In [ ]:
#prediction_GBR,pd.DataFrame(prediction_GBR).describe()

In [ ]:
#submission = pd.DataFrame({'id':sample_submission['id'],
                           #'visitors':prediction_GBR})
#submission.to_csv('submission_GBR.csv',index=False)

In [ ]:
KNR.fit(train_x,np.log1p(train_y))
prediction_KNR = np.expm1(KNR.predict(test_x))
rmsle_score_KNR = rmsle(test_y,prediction_KNR)

In [ ]:
prediction_KNR,pd.DataFrame(prediction_KNR).describe()

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':prediction_KNR})
submission.to_csv('submission_KNR.csv',index=False)

In [ ]:
#submission = pd.DataFrame({'id':sample_submission['id'],
#                           'visitors':(prediction_KNR+prediction_GBR+prediction)/3})
#submission.to_csv('submission_ensemble.csv',index=False)

In [ ]:
#submission.describe()

In [ ]:
ETR = ensemble.ExtraTreesRegressor(n_estimators=1000,n_jobs=4,max_depth=10,bootstrap=True)
RFR = ensemble.RandomForestRegressor(n_estimators=1000,n_jobs=4,max_depth=10)

In [ ]:
ETR.fit(train_x,np.log1p(train_y))
prediction_ETR = np.expm1(ETR.predict(test_x))
rmsle_score_ETR = rmsle(test_y,prediction_ETR)

In [ ]:
prediction_ETR

In [ ]:
ETR.feature_importances_

In [ ]:
train_x.columns

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':prediction_ETR})
submission.to_csv('submission_ETR.csv',index=False)

In [ ]:
RFR.fit(train_x,np.log1p(train_y))
prediction_RFR = np.expm1(RFR.predict(test_x))
rmsle_score_RFR = rmsle(test_y,prediction_RFR)

In [ ]:
RFR.feature_importances_

In [ ]:
prediction_RFR

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':prediction_RFR})
submission.to_csv('submission_RFR.csv',index=False)

In [ ]:
#sub1 = pd.read_csv('submission_XGB_0.506.csv')
#sub2 = pd.read_csv('submission_KNR_0.503.csv')
#sub3 = pd.read_csv('submission_GBR_0.509.csv')
#sub4 = pd.read_csv('submission_RFR_0.507.csv')
#sub5 = pd.read_csv('submission_ETR_0.51.csv')
sub1 = pd.read_csv('submission_XGB.csv')
sub2 = pd.read_csv('submission_KNR.csv')
sub3 = pd.read_csv('submission_GBR.csv')
sub4 = pd.read_csv('submission_RFR.csv')
sub5 = pd.read_csv('submission_ETR.csv')

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],
                           'visitors':(sub1['visitors']+sub2['visitors']+sub3['visitors']+sub4['visitors']+sub5['visitors'])/5})
submission.to_csv('submission_ensemble_5.csv',index=False)

In [ ]:
submission['visitors'].describe()
print(rmsle_score_xgb)
print(rmsle_score_KNR)
print(rmsle_score_ETR)
print(rmsle_score_RFR)


In [ ]:
rmsle_score_ESS = rmsle(test_y, submission['visitors'])
print(rmsle_score_ESS)                 
#sub6 = pd.read_csv('C:\\Users\\Leo\\OneDrive\\air_reserve\\submission_ensemble_5AVG_0.491.csv')
#sub7 = pd.read_csv('C:\\Users\\Leo\\OneDrive\\air_reserve\\submission_0.479.csv')
#sub8 = pd.read_csv('C:\\Users\\Leo\\OneDrive\\air_reserve\\submission_h2o_0.48.csv')

In [ ]:
#submission = pd.DataFrame({'id':sample_submission['id'],
#                           'visitors':sub6['visitors']*0.1+sub7['visitors']*0.8+sub8['visitors']*0.1})
#submission.to_csv('submission_ensemble_3.csv',index=False)